## Filtragem arquivos transparência - Saúde

In [ ]:
import os
import glob
import openpyxl
import pandas as pd

In [ ]:
def listar_arquivos(diretorio='Favorecidos', extensao='xlsx'):
    search = "input/{}/*.{}".format(diretorio, extensao)
    files = glob.glob(search)
    files.sort()
    return files

In [ ]:
arquivos = listar_arquivos('Contratos')

In [ ]:
arquivos

In [ ]:
dff = pd.DataFrame()
for arquivo in arquivos:
    df = pd.read_excel(arquivo, sheet_name='Plan1')
    # SECRETARIA MUNICIPAL DE SAÚDE = 1800 e FUNDO DE ASSISTÊNCIA À SAÚDE = 1333
    df = df[(df['Órgão executor'] == 1800) | (df['Órgão executor'] == 1333)]
    dff = dff.append(df)

In [ ]:
len(dff)

In [ ]:
len(dff['Código favorecido'].unique())

In [ ]:
aggregations = {
    'Favorecido': 'first',
    'Valor Instrumento': 'sum',
    'Valor do acréscimo ou redução': 'sum',
    'Saldo do instrumento': 'sum',
    'Saldo executado': 'sum',
    'Valor empenhado': 'sum',
    'Valor liquidado': 'sum',
    'Valor pago': 'sum'
}
dff1 = dff.groupby(['Ano instrumento', 'Código favorecido']).agg(aggregations)

In [ ]:
dff1

In [ ]:
aggregations = {
    'Valor pago': 'sum',
    'Código favorecido': 'count'
}
dff2 = dff.groupby('Ano instrumento').agg(aggregations)

In [ ]:
dff2

In [ ]:
with pd.ExcelWriter('output/gastos_totais_saude.xlsx') as writer:
    dff1.to_excel(writer, sheet_name='detalhado')
    dff2.to_excel(writer, sheet_name='anual')

In [ ]:
list(dff)

In [ ]:
codigos = [
    2385669000174,
    9652823000176,
    61699567000192,
    24232886000167,
    343941000128,
    8882456000134,
    33981408000140,
    27774561000102,
    31104896000182,
    33005638000174,
    6942340000118,
    5153789000134,
    2833927377000140,
    42375097000167,
    48211585000115,
    5696218000146,
    2158304200084,
    7345851000115,
    33722224000166,
    7813739000161,
    10635117000103,
    66518267000183
]

In [ ]:
dff = dff[dff['Código favorecido'].isin(codigos)]

In [ ]:
len(dff)

In [ ]:
aggregations = {
    'Favorecido': 'first',
    'Valor Instrumento': 'sum',
    'Valor do acréscimo ou redução': 'sum',
    'Saldo do instrumento': 'sum',
    'Saldo executado': 'sum',
    'Valor empenhado': 'sum',
    'Valor liquidado': 'sum',
    'Valor pago': 'sum'
}
dff3 = dff.groupby(['Ano instrumento', 'Código favorecido']).agg(aggregations)

In [ ]:
dff3

In [ ]:
aggregations = {
    'Valor pago': 'sum',
    'Código favorecido': 'count'
}
dff4 = dff.groupby('Ano instrumento').agg(aggregations)

In [ ]:
dff4

In [ ]:
with pd.ExcelWriter('output/gastos_os_saude.xlsx') as writer:
    dff3.to_excel(writer, sheet_name='detalhado')
    dff4.to_excel(writer, sheet_name='anual')